In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import re
from datetime import datetime

In [2]:
def  scrapping_oai_sets_dewey():
    url = "https://www.theses.fr/schemas/tef/recommandation/oai_sets.html"
    resp = requests.get(url).text  # ou f = http.request('GET', url).data
    soup = BeautifulSoup(resp, features="lxml")
    oai_list = []
    for row in soup.findAll("table")[0].findAll("tr"):
        label = re.sub('<!--.*-->|\r|\n', '', str(row.findAll("td")[0].get_text(strip=True)), flags=re.DOTALL)
        label = re.sub('\s{2,}|&nbsp;', ' ', label)
        oai_list.append(
            {
                "label": label,
                "code": row.findAll("td")[1].get_text(strip=True),
            }
        )
    df = pd.DataFrame(oai_list[1:])
    df['main_domain'] = df['code'].apply(lambda x: 'Sciences, Technologies, Santé' if ((str(x[4]) == "5") | (str(x[4]) == "6") | (str(x[4:7]) == "004")) else 'Lettres, sciences Humaines et Sociales')
    return df.to_dict('records')

In [3]:
oai = scrapping_oai_sets_dewey()
oai

[{'label': 'Informatique, information, généralités',
  'code': 'ddc:000',
  'main_domain': 'Lettres, sciences Humaines et Sociales'},
 {'label': 'Informatique',
  'code': 'ddc:004',
  'main_domain': 'Sciences, Technologies, Santé'},
 {'label': "Bibliothéconomie et sciences de l'information",
  'code': 'ddc:020',
  'main_domain': 'Lettres, sciences Humaines et Sociales'},
 {'label': 'Organisations générales et muséologie',
  'code': 'ddc:060',
  'main_domain': 'Lettres, sciences Humaines et Sociales'},
 {'label': "Médias d'information, journalisme, édition",
  'code': 'ddc:070',
  'main_domain': 'Lettres, sciences Humaines et Sociales'},
 {'label': 'Manuscrits et livres rares',
  'code': 'ddc:090',
  'main_domain': 'Lettres, sciences Humaines et Sociales'},
 {'label': 'Philosophie, psychologie',
  'code': 'ddc:100',
  'main_domain': 'Lettres, sciences Humaines et Sociales'},
 {'label': 'Métaphysique',
  'code': 'ddc:110',
  'main_domain': 'Lettres, sciences Humaines et Sociales'},
 {'la

In [38]:
oai[oai["code"] == "ddc:340"]["main_domain"]

'30    Lettres, sciences Humaines et Sociales\nName: main_domain, dtype: object'

In [46]:
df = pd.read_csv("data/03_primary/2022-05-30/theses_uca_processed.csv",sep=",",encoding="utf-8")
df

,accessible,auteurs_0_idref,auteurs_0_nom,auteurs_0_prenom,cas,code_etab,date_soutenance,directeurs_these_0_idref,directeurs_these_0_nom,directeurs_these_0_prenom,...,embargo_duree,has_exist_embargo,oai_set_specs_0,oai_set_specs_1,oai_set_specs_0_label,oai_set_specs_1_label,oai_set_specs_0_regroup,oai_set_specs_0_regroup_label,oai_set_specs_1_regroup,oai_set_specs_1_regroup_label
0,non,191342505,Labonia,Mónica,cas5,NICE,2014-10-03,03464301X,Candau,Joël,...,0,non,ddc:300,NaN,"Sciences sociales, sociologie, anthropologie",NaN,ddc:300,"Sciences sociales, sociologie, anthropologie",NaN,NaN
1,oui,182472027,Lanzara,Delphine,cas1,NICE,2014-11-25,056260911,Strickler,Yves,...,0,non,ddc:340,NaN,Droit,NaN,ddc:300,"Sciences sociales, sociologie, anthropologie",NaN,NaN
2,non,191343811,Ngo Koy,Hermine Odette,cas5,NICE,2014-07-17,055794068,Gourdet,Geneviève,...,0,non,ddc:340,NaN,Droit,NaN,ddc:300,"Sciences sociales, sociologie, anthropologie",NaN,NaN
3,oui,142596965,Tollinchi,Alexandre-Guillaume,cas1,NICE,2015-11-20,117616842,Siiriainen,Fabrice,...,1138,oui,ddc:340,NaN,Droit,NaN,ddc:300,"Sciences sociales, sociologie, anthropologie",NaN,NaN
4,oui,189075481,Trucco,Daniela,cas1,NICE,2015-09-23,050249037,Savarese,Éric,...,0,non,ddc:320,NaN,Science politique,NaN,ddc:300,"Sciences sociales, sociologie, anthropologie",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7847,oui,260344079,Al Zoobi,Ali,cas1,COAZ,2021-11-25,074539043,Coudert,David,...,51,oui,ddc:004,NaN,Informatique,NaN,ddc:000,"Informatique, information, généralités",NaN,NaN
7848,oui,260342939,Iecker Ricardo,Guilherme,cas1,COAZ,2021-09-09,18310966X,Neglia,Giovanni,...,0,non,ddc:004,NaN,Informatique,NaN,ddc:000,"Informatique, information, généralités",NaN,NaN
7849,oui,260347655,Laversa,Laetitia,cas1,COAZ,2021-12-14,104491957,Lozes,Etienne,...,0,non,ddc:004,NaN,Informatique,NaN,ddc:000,"Informatique, information, généralités",NaN,NaN
7850,oui,260418773,Bhole,Ashish,cas1,COAZ,2021-11-17,120046059,Nkonga,Boniface,...,0,non,ddc:510,NaN,Mathématiques,NaN,ddc:500,Sciences de la nature et mathématiques,NaN,NaN


In [47]:
df1 = pd.merge(df,oai, left_on='oai_set_specs_0', right_on='code',how="left").drop(columns=['label','code']).rename(columns={'main_domain': 'oai_set_specs_0_main_domain'})
df1

,accessible,auteurs_0_idref,auteurs_0_nom,auteurs_0_prenom,cas,code_etab,date_soutenance,directeurs_these_0_idref,directeurs_these_0_nom,directeurs_these_0_prenom,...,has_exist_embargo,oai_set_specs_0,oai_set_specs_1,oai_set_specs_0_label,oai_set_specs_1_label,oai_set_specs_0_regroup,oai_set_specs_0_regroup_label,oai_set_specs_1_regroup,oai_set_specs_1_regroup_label,oai_set_specs_0_main_domain
0,non,191342505,Labonia,Mónica,cas5,NICE,2014-10-03,03464301X,Candau,Joël,...,non,ddc:300,NaN,"Sciences sociales, sociologie, anthropologie",NaN,ddc:300,"Sciences sociales, sociologie, anthropologie",NaN,NaN,"Lettres, sciences Humaines et Sociales"
1,oui,182472027,Lanzara,Delphine,cas1,NICE,2014-11-25,056260911,Strickler,Yves,...,non,ddc:340,NaN,Droit,NaN,ddc:300,"Sciences sociales, sociologie, anthropologie",NaN,NaN,"Lettres, sciences Humaines et Sociales"
2,non,191343811,Ngo Koy,Hermine Odette,cas5,NICE,2014-07-17,055794068,Gourdet,Geneviève,...,non,ddc:340,NaN,Droit,NaN,ddc:300,"Sciences sociales, sociologie, anthropologie",NaN,NaN,"Lettres, sciences Humaines et Sociales"
3,oui,142596965,Tollinchi,Alexandre-Guillaume,cas1,NICE,2015-11-20,117616842,Siiriainen,Fabrice,...,oui,ddc:340,NaN,Droit,NaN,ddc:300,"Sciences sociales, sociologie, anthropologie",NaN,NaN,"Lettres, sciences Humaines et Sociales"
4,oui,189075481,Trucco,Daniela,cas1,NICE,2015-09-23,050249037,Savarese,Éric,...,non,ddc:320,NaN,Science politique,NaN,ddc:300,"Sciences sociales, sociologie, anthropologie",NaN,NaN,"Lettres, sciences Humaines et Sociales"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7847,oui,260344079,Al Zoobi,Ali,cas1,COAZ,2021-11-25,074539043,Coudert,David,...,oui,ddc:004,NaN,Informatique,NaN,ddc:000,"Informatique, information, généralités",NaN,NaN,"Sciences, Technologies, Santé"
7848,oui,260342939,Iecker Ricardo,Guilherme,cas1,COAZ,2021-09-09,18310966X,Neglia,Giovanni,...,non,ddc:004,NaN,Informatique,NaN,ddc:000,"Informatique, information, généralités",NaN,NaN,"Sciences, Technologies, Santé"
7849,oui,260347655,Laversa,Laetitia,cas1,COAZ,2021-12-14,104491957,Lozes,Etienne,...,non,ddc:004,NaN,Informatique,NaN,ddc:000,"Informatique, information, généralités",NaN,NaN,"Sciences, Technologies, Santé"
7850,oui,260418773,Bhole,Ashish,cas1,COAZ,2021-11-17,120046059,Nkonga,Boniface,...,non,ddc:510,NaN,Mathématiques,NaN,ddc:500,Sciences de la nature et mathématiques,NaN,NaN,"Sciences, Technologies, Santé"


In [48]:
df2 = pd.merge(df1,oai, left_on='oai_set_specs_1', right_on='code',how="left").drop(columns=['label','code']).rename(columns={'main_domain': 'oai_set_specs_1_main_domain'})
df2

,accessible,auteurs_0_idref,auteurs_0_nom,auteurs_0_prenom,cas,code_etab,date_soutenance,directeurs_these_0_idref,directeurs_these_0_nom,directeurs_these_0_prenom,...,oai_set_specs_0,oai_set_specs_1,oai_set_specs_0_label,oai_set_specs_1_label,oai_set_specs_0_regroup,oai_set_specs_0_regroup_label,oai_set_specs_1_regroup,oai_set_specs_1_regroup_label,oai_set_specs_0_main_domain,oai_set_specs_1_main_domain
0,non,191342505,Labonia,Mónica,cas5,NICE,2014-10-03,03464301X,Candau,Joël,...,ddc:300,NaN,"Sciences sociales, sociologie, anthropologie",NaN,ddc:300,"Sciences sociales, sociologie, anthropologie",NaN,NaN,"Lettres, sciences Humaines et Sociales",NaN
1,oui,182472027,Lanzara,Delphine,cas1,NICE,2014-11-25,056260911,Strickler,Yves,...,ddc:340,NaN,Droit,NaN,ddc:300,"Sciences sociales, sociologie, anthropologie",NaN,NaN,"Lettres, sciences Humaines et Sociales",NaN
2,non,191343811,Ngo Koy,Hermine Odette,cas5,NICE,2014-07-17,055794068,Gourdet,Geneviève,...,ddc:340,NaN,Droit,NaN,ddc:300,"Sciences sociales, sociologie, anthropologie",NaN,NaN,"Lettres, sciences Humaines et Sociales",NaN
3,oui,142596965,Tollinchi,Alexandre-Guillaume,cas1,NICE,2015-11-20,117616842,Siiriainen,Fabrice,...,ddc:340,NaN,Droit,NaN,ddc:300,"Sciences sociales, sociologie, anthropologie",NaN,NaN,"Lettres, sciences Humaines et Sociales",NaN
4,oui,189075481,Trucco,Daniela,cas1,NICE,2015-09-23,050249037,Savarese,Éric,...,ddc:320,NaN,Science politique,NaN,ddc:300,"Sciences sociales, sociologie, anthropologie",NaN,NaN,"Lettres, sciences Humaines et Sociales",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7847,oui,260344079,Al Zoobi,Ali,cas1,COAZ,2021-11-25,074539043,Coudert,David,...,ddc:004,NaN,Informatique,NaN,ddc:000,"Informatique, information, généralités",NaN,NaN,"Sciences, Technologies, Santé",NaN
7848,oui,260342939,Iecker Ricardo,Guilherme,cas1,COAZ,2021-09-09,18310966X,Neglia,Giovanni,...,ddc:004,NaN,Informatique,NaN,ddc:000,"Informatique, information, généralités",NaN,NaN,"Sciences, Technologies, Santé",NaN
7849,oui,260347655,Laversa,Laetitia,cas1,COAZ,2021-12-14,104491957,Lozes,Etienne,...,ddc:004,NaN,Informatique,NaN,ddc:000,"Informatique, information, généralités",NaN,NaN,"Sciences, Technologies, Santé",NaN
7850,oui,260418773,Bhole,Ashish,cas1,COAZ,2021-11-17,120046059,Nkonga,Boniface,...,ddc:510,NaN,Mathématiques,NaN,ddc:500,Sciences de la nature et mathématiques,NaN,NaN,"Sciences, Technologies, Santé",NaN


In [51]:
df3 = df2[df2.oai_set_specs_1_label.notna()]

In [54]:
df3[df3.oai_set_specs_1_label != df3.oai_set_specs_0_label]

,accessible,auteurs_0_idref,auteurs_0_nom,auteurs_0_prenom,cas,code_etab,date_soutenance,directeurs_these_0_idref,directeurs_these_0_nom,directeurs_these_0_prenom,...,oai_set_specs_0,oai_set_specs_1,oai_set_specs_0_label,oai_set_specs_1_label,oai_set_specs_0_regroup,oai_set_specs_0_regroup_label,oai_set_specs_1_regroup,oai_set_specs_1_regroup_label,oai_set_specs_0_main_domain,oai_set_specs_1_main_domain
7,non,057479690,Beaudet,Pascale,cas5,NICE,1999-01-01,026705028,Bardolph,Jacqueline,...,ddc:420,ddc:820,Langue anglaise. Anglo-saxon,Littératures anglaise et anglo-saxonne,ddc:400,Langues et linguistique,ddc:800,"Histoire et critique littéraires, rhétorique","Lettres, sciences Humaines et Sociales","Lettres, sciences Humaines et Sociales"
43,non,082823995,Lombardi,Eric,cas5,NICE,1996-01-01,030489083,Iooss,Gérard,...,ddc:500,ddc:600,Sciences de la nature et mathématiques,Technologie (Sciences appliquées),ddc:500,Sciences de la nature et mathématiques,ddc:600,Technologie (Sciences appliquées),"Sciences, Technologies, Santé","Sciences, Technologies, Santé"
44,non,03362626X,Chiaramello,Jean-Marc,cas5,NICE,1988-01-01,176945911,Cambon,Aimé,...,ddc:530,ddc:540,Physique,"Chimie, minéralogie, cristallographie",ddc:500,Sciences de la nature et mathématiques,ddc:500,Sciences de la nature et mathématiques,"Sciences, Technologies, Santé","Sciences, Technologies, Santé"
51,non,069241589,Fernandez,Gérard,cas5,NICE,1989-01-01,032703155,Peyret,Roger,...,ddc:510,ddc:620,Mathématiques,Sciences de l'ingénieur,ddc:500,Sciences de la nature et mathématiques,ddc:600,Technologie (Sciences appliquées),"Sciences, Technologies, Santé","Sciences, Technologies, Santé"
54,non,130322547,Nicolle,Benjamin,cas5,NICE,2008-01-01,110356004,Jacquemod,Gilles,...,ddc:530,ddc:620,Physique,Sciences de l'ingénieur,ddc:500,Sciences de la nature et mathématiques,ddc:600,Technologie (Sciences appliquées),"Sciences, Technologies, Santé","Sciences, Technologies, Santé"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7808,non,260225851,Bonnet,Raphaël,cas5,COAZ,2021-09-27,074145061,Peyron,Jean-François,...,ddc:570,ddc:610,"Sciences de la vie, biologie, biochimie",Médecine et santé,ddc:500,Sciences de la nature et mathématiques,ddc:600,Technologie (Sciences appliquées),"Sciences, Technologies, Santé","Sciences, Technologies, Santé"
7816,oui,258009071,Siciak,Antonin,cas1,COAZ,2021-07-05,066972140,Kaiser,Robin,...,ddc:530,ddc:520,Physique,"Astronomie, cartographie, géodésie",ddc:500,Sciences de la nature et mathématiques,ddc:500,Sciences de la nature et mathématiques,"Sciences, Technologies, Santé","Sciences, Technologies, Santé"
7819,oui,258222441,Macquet,Adrian,cas1,COAZ,2021-10-15,231950314,Bizouard,Marie-Anne,...,ddc:520,ddc:530,"Astronomie, cartographie, géodésie",Physique,ddc:500,Sciences de la nature et mathématiques,ddc:500,Sciences de la nature et mathématiques,"Sciences, Technologies, Santé","Sciences, Technologies, Santé"
7829,oui,260259233,Guigui,Nicolas,cas1,COAZ,2021-11-18,035316489,Pennec,Xavier,...,ddc:004,ddc:620,Informatique,Sciences de l'ingénieur,ddc:000,"Informatique, information, généralités",ddc:600,Technologie (Sciences appliquées),"Sciences, Technologies, Santé","Sciences, Technologies, Santé"


In [27]:
df[df["code"] == "ddc:820"]

,label,code,main_domain,dewey
79,Littératures anglaise et anglo-saxonne,ddc:820,"Lettres, sciences Humaines et Sociales",82


In [10]:
import os
os.listdir("data")

['01_raw',
 '02_intermediate',
 '03_primary',
 '04_feature',
 '05_model_input',
 '06_models',
 '07_model_output',
 '08_reporting']

In [1]:
import os
flows_py = []
for root, dirs, files in os.walk("C:/Users/geoffroy/Docker/prefect-flows/flows"):
    for file in files:
        if file == "flow.py":
            flows_py.append(os.path.join(root, file))
flows_py

['C:/Users/geoffroy/Docker/prefect-flows/flows\\bso_publis_scopus\\flow.py',
 'C:/Users/geoffroy/Docker/prefect-flows/flows\\bso_theses\\flow.py']